# Installing the HASP/HSLA Spectral-Combination Script

### <span style="font-weight:normal">This Notebook is designed to walk you through downloading, installing, and running the spectral-combination script used by the **Hubble Advanced Spectral Products (HASP)** and the **Hubble Spectral Legacy Archive (HSLA)**.</span>

## Learning Goals: 

By the end of this tutorial, you will: 

* Learn how to download `micromamba` and create the `stenv` environment

* Learn how to download and install the HASP/HSLA co-add script

* Learn how to run the script

* Understand the naming conventions of the co-added output files


## Table of Contents
**0. [Introduction](#introduction)**

**1. [Downloading and Installing `micromamba`](#conda)**

\- 1.1 [Installing `micromamba`](#installconda)

\- 1.2 [Creating the `stenv` Environment](#createstenv)

**2. [Downloading and Installing the HASP/HSLA Script](#downloadcode)**

**3. [Running the Co-add Script](#runscript)**

\- 3.1 [Retrieving the Data](#retrievedata)

\- 3.2 [Running the Script](#runscript)

\- 3.3 [Understanding the Output Files](#inspectoutput)


<a id = introduction></a>
## 0. Introduction

The Hubble Advanced Spectral Products (HASP) and Hubble Spectroscopic Legacy Archive (HSLA) combine data taken with the Space Telescope Imaging Spectrograph (STIS) and the Cosmic Origins Spectrograph (COS) aboard the Hubble Space Telescope (HST) into high-quality spectra with broad wavelength coverage (from the ultraviolet to the near-infrared if the data exist) that are publicly available to the scientific community.  HASP combines all of the STIS and COS data for each target within a single program.  HSLA combines all of the STIS and COS data in the MAST archive -- across multiple programs -- into a single spectrum for each target.

The same script is used by both projects; it is available for use by observers who wish to produce custom products beyond those provided by HASP and HSLA through MAST.  Briefly, the code co-adds all of the spectra taken with the same grating, then splices together (or abuts) the spectra from multiple gratings into a single spectrum with the broadest possible wavelength range.  In particular, the script combines multiple orders from the STIS echelle modes and multiple central-wavelength settings for a single STIS or COS grating, tasks that that the current STIS and COS pipelines do not perform.  For a detailed explanation of the logic used to combine spectra, see the [HASP](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/cos/documentation/instrument-science-reports-isrs/_documents/ISR2024-01.pdf) and [HSLA](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/cos/documentation/instrument-science-reports-isrs/_documents/ISR2025-18.pdf) ISRs.

This notebook focuses on the installation of the co-add script and provides a quick overview of its use. To see an example of downloading COS and STIS datasets, running the script, and analyzing the output, please review the [HASP Coadd Tutorial notebook](https://spacetelescope.github.io/hst_notebooks/notebooks/HASP/CoaddTutorial/CoaddTutorial.html).  For an overview of the HSLA products, see the notebook, [Introduction to the HSLA Data Products and Tools](https://github.com/spacetelescope/hst_notebooks/tree/main/notebooks/HSLA/Intro/Intro.ipynb).

<a id = conda></a>
## 1. Downloading and Installing `micromamba`

<a id = installconda></a>
### 1.1 Installing `micromamba`

`Conda` is a package, dependency, and environment manager that runs on Windows, Mac, and Linux. `Conda` allows us to install Python packages and create isolated conda environments, each with its own packages and dependencies. By switching between `conda` environments, we avoid conflicts between different versions of packages.  `mamba` is a rewrite of conda that is much faster at resolving dependencies with near-parity of commands. `micromamba` is `mamba` packaged into a single binary, which makes installation and maintenance much easier. For these reasons, we recommended that you use `micromamba`.

We will create a `micromamba` environment to install the packages needed for the HASP/HSLA script. First check whether `micromamba` is already installed by running this command:

In [1]:
!micromamba --version

2.3.3


Running the command should print out the version of `micromamba` that you have installed (in this case, `micromamba 2.3.0`). If the command returns a message saying that the package is unknown, then `micromamba` is not installed on your machine. 

To install it, follow the directions for `micromamba` on the [`Getting Started`](https://stenv.readthedocs.io/en/latest/getting_started.html) page for `stenv`.  There are a couple of options.  One is to enter the following in a terminal window:

```
"${SHELL}" <(curl -L micro.mamba.pm/install.sh)
```

When you're done, run the version command again to confirm that micromamba is installed.

<a id = createstenv></a>
### 1.2 Creating the `stenv` Environment

The next step is to install the `stenv` environment, which contains all of the software packages used by the HASP/HSLA software.  Continue to follow the instructions on the [`Getting Started`](https://stenv.readthedocs.io/en/latest/getting_started.html) page for `stenv`.  It will direct you to the [`Releases`](https://github.com/spacetelescope/stenv/releases) page, from which you must retrieve the yaml file appropriate for your system.  Install the environment using the command

In [2]:
!micromamba env create --name stenv --file ~/Downloads/stenv-Linux-X64-py3.13-2025.07.16.yaml

info     libmamba ****************** Backtrace Start ******************
debug    libmamba Loading configuration
trace    libmamba Compute configurable 'create_base'
trace    libmamba Compute configurable 'no_env'
trace    libmamba Compute configurable 'no_rc'
trace    libmamba Compute configurable 'rc_files'
trace    libmamba Compute configurable 'root_prefix'
trace    libmamba Get RC files configuration from locations up to HomeDir
trace    libmamba Configuration found at '/home/runner/work/_temp/setup-micromamba/.condarc'
trace    libmamba Update configurable 'no_env'
trace    libmamba Compute configurable 'envs_dirs'
trace    libmamba Compute configurable 'file_specs'
error    libmamba YAML spec file '/home/runner/Downloads/stenv-Linux-X64-py3.13-2025.07.16.yaml' not found
critical libmamba File not found. Aborting.
info     libmamba ****************** Backtrace End ********************


(Replace the file name with that of the yaml file that you've just retrieved.)  This will take a few minutes.  Activate the new environment using the command

In [3]:
!micromamba activate stenv

Your parent process name is Name:	python.
If your shell is xonsh, please use "-s xonsh".

'micromamba' is running as a subprocess and can't modify the parent shell.
Thus you must initialize your shell before using activate and deactivate.

To initialize the current bash shell, run:
    $ eval "$(micromamba shell hook --shell bash)"
and then activate or deactivate with:
    $ micromamba activate
To automatically initialize all future (bash) shells, run:
    $ micromamba shell init --shell bash --root-prefix=~/.local/share/mamba
If your shell was already initialized, reinitialize your shell with:
    $ micromamba shell reinit --shell bash
Otherwise, this may be an issue. In the meantime you can run commands. See:
    $ micromamba run --help

Supported shells are {bash, zsh, csh, posix, xonsh, cmd.exe, powershell, fish, nu}.
critical libmamba Shell not initialized


Now that you have created and activated an `stenv` environment, you can download the co-add script.

<a id = downloadcode></a>
## 2. Downloading and Installing the HASP/HSLA Script

Download the HASP/HSLA co-add script from [the HASP Github repository](https://github.com/spacetelescope/hasp).

**You only need to run one of the two options below, depending on your computer and Git settings.**  (If you have no idea which option to choose, try Option A.  It will probably work.)

#### Option A, using `pip`:

`pip install` clones the repository and installs the packages according to the setup configuration of the repository. You may need to create a Personal Access Token (PAT) to use instead of your Github account's password, since support for password authentication was removed in 2021. You can learn how to create a PAT [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token).

In [4]:
!pip install git+https://github.com/spacetelescope/hasp.git

  Cloning https://github.com/spacetelescope/hasp.git to /tmp/pip-req-build-eqi6go6y
  Running command git clone --filter=blob:none --quiet https://github.com/spacetelescope/hasp.git /tmp/pip-req-build-eqi6go6y


  Resolved https://github.com/spacetelescope/hasp.git to commit e0f5e8c82a45c0145dbb23579097647409dd0778


  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


  Preparing metadata (pyproject.toml) ... -

 done


#### Option B, using `git clone`:

While `git clone` also downloads the repository, it differs from `pip` in that it also downloads all of the history and files of the repository. If you have a password-protected SSH key, rather than a PAT, you can clone the repo by running the following code in your terminal. If you want to setup a SSH key, you can learn how to do so [here](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account).

In [5]:
!git clone git@github.com:spacetelescope/hasp.git

Cloning into 'hasp'...



fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


This will install additional dependencies using the `pyproject.toml` file. 

#### After you've downloaded the code using Option A or B, run this command to download the script's dependencies:

In [6]:
!pip install hasp

<a id = runscript></a>
## 3. Running the Co-add Script

<a id = retrievedata></a>
### 3.1 Retrieving the Data

The program operates on the following COS and STIS files:

*    [X1D](https://hst-docs.stsci.edu/cosdhb/chapter-2-cos-data-files/2-4-cos-data-products#:~:text=in%20the%20association.-,One%2DDimensional%20Extracted%20Spectra%20(x1d%2C%20x1dsum),-The%20COS%20pipeline) - the one-dimensional extracted product spectra.
  
*    [SX1](https://hst-docs.stsci.edu/stisdhb/chapter-2-stis-data-structure/2-2-types-of-stis-files#:~:text=corrected%20imaging%20data.-,_sx1,-table) - the one-dimensional extracted spectra from combined or cosmic-ray rejected images. This file is produced only for STIS data. 

Make sure that all of these files, for every spectrum that you wish to combine, are in the same directory. The script will combine only the files in this directory.

<a id = runscript></a>
### 3.2 Running the Script

Now that you have retrieved your data, you can run the co-add script.  HASP (which does not combine data from multiple programs) and HSLA (which does) run the same script.  The difference is that HSLA calls the program with the -x argument, thus instructing the code to perform cross-program co-adds.

| Command-line Option | Value |
|----------|----------|
| `--input_directory` (`-i`) | The path to the directory that contains the data to be co-added (_required_)|
| `--output_directory` (`-o`) | The path to the directory that will contain the newly co-added products (_required_) |
| `--clobber` (`-o`) | If set, overwrite existing products |
| `--cross_program` (`-x`) | Create cross-program (HSLA) coadds (_optional_) |
| `--threshold` (`-t`) | The threshold for flux-based filtering (_optional; default is -50_) |
| `--snrmax` (`-s`) | The maximum SNR for flux-based filtering (_optional; default is 20_) |
| `--grating_table` (`-g`) | Name of custom grating priority table (_optional_) |
| `--no_keyword_filtering` (`-k`) | Disable keyword-based filtering (except for STIS PRISM data, which is always filtered) (_optional_) |

**Note:** If you wish to exclude any data files from the co-add, simply remove them from the input directory before running the script.

To run the HASP version of the code, open your terminal, activate your `micromamba` environment, and enter:

```
swrapper -i <YOUR_DATA_DIR> -o <YOUR_PRODUCT_DIR>
```

The co-added spectra will be deposited in the output directory.

<a id = inspectoutput></a>
### 3.3 Understanding the HASP Output Files

The HSLA script produces four sets of files for each target. First are the co-added spectra for each grating and visit:

`hst_programID_instrument_targetname_grating_obset_cspec.fits`, where `obset` is a six-digit string like `O4A520`

Second are the co-added spectra for each grating in the program:

`hst_programID_instrument_targetname_grating_obset_cspec.fits`, where `obset` is a four-digit string like `O4A5`

Third are the abutted spectra from multiple gratings for each visit:

`hst_programID_instrument_targetname_allGratings_obset_cspec.fits`

Fourth are the abutted spectra from multiple gratings for the entire program:

`hst_programID_instrument_targetname_allGratings_cspec.fits`

Here's an example. These are the output files for a STIS GD71 dataset that is co-added in the [HASP Coadd Tutorial notebook](https://spacetelescope.github.io/hst_notebooks/notebooks/HASP/CoaddTutorial/CoaddTutorial.html). Here, the `programID` is `7656`, the `instrument` is `STIS`, and the `targetname` is `gd71`.

| Set | Filename | Description |
|----------|----------|----------|
| 1 | `hst_7656_stis_gd71_sg140l_o4a520_cspec.fits` | Co-adding all `G140L` observations for visit `O4A520` |
| 2 | `hst_7656_stis_gd71_sg140l_o4a5_cspec.fits` | Co-adding all `G140L` spectra for program `04A5` |
| 3 | `hst_7656_stis_gd71_sg140l-sg230l-g430l-g750l_o4a520_cspec.fits` | Abutting multiple gratings for visit `O4A520` |
| 4 | `hst_7656_stis_gd71_sg140l-sg230l-g430l-g750l_o4a5_cspec.fits` | Abutting multiple gratings for all visits in program `O4A5` |

HST file naming conventions use a combination of three letters and numbers to create a unique association between a PI's proposal ID and program ID, meaning that `o4a5` is essentially the program ID for our example. More information is available on the [MAST HST file naming convention page](https://archive.stsci.edu/hlsp/ipppssoot.html). 

For an overview of the output files from the HSLA version of the code, please see the notebook, [Introduction to the HSLA Data Products and Tools](https://github.com/spacetelescope/hst_notebooks/tree/main/notebooks/HSLA/Intro/Intro.ipynb).

## About this Notebook
**Authors:** Sierra Gomez (sigomez@stsci.edu) and Van Dixon (dixon@stsci.edu)

**Updated on:** 10/27/2025

*This tutorial was generated to be in compliance with the [STScI style guides](https://github.com/spacetelescope/style-guides).  We would like to cite the [Jupyter guide](https://github.com/spacetelescope/style-guides/blob/master/templates/example_notebook.ipynb) in particular.*


<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>